In [1]:
import json, os, glob
import random
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import xgboost as xgb
import optuna
from data_preparation.dataset_statewise import YDataset
import torch
import pickle as pkl
from data_preparation.utils import evalMetrics

## prepare data

In [2]:

# trucate time series
start_doy_idx = 11 
end_doy_idx = 38
kernel=5
feature_idx = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 14]

npy_path = '/app/dev/spatial_encoding/data/composite_npy'
label_path= '/app/dev/spatial_encoding/data/composite_npy/labels.json'
norm_path = '/app/dev/spatial_encoding/2024_08/data_preparation/min_max_L2_U98_hist.json'

# years
train_years = list(range(2003, 2020))
test_years_d = [2020]
test_years_nd = [2021]

## support vector regression

In [7]:
def svr_train(seedlist, num_trial, out_dir, exp_name='svr'):
    
    # prepare output_dir 
    os.makedirs(out_dir, exist_ok=True)
    exp_params = {}
    
    # make directory for experiment
    output_dir = os.path.join(out_dir, exp_name)
    os.makedirs(output_dir, exist_ok=True)

    # -- collector for metrics
    metrics_train = []
    metrics_val = []
    metrics_test = []   

    for s in seedlist:
        np.random.seed(s)
        random.seed(s)
        torch.manual_seed(s)

        # Initialize YieldDataset with various parameters
        train_dataset = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=train_years, mode='train', seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # Initialize YieldDataset with various parameters
        val_dataset = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=train_years, mode='validation', seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # Initialize YieldDataset with various parameters
        test_dataset_d = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=test_years_d, mode=None, seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        test_dataset_nd = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=test_years_nd, mode=None, seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # ----- create array for experiment settings
        X_t, y_t  = train_dataset.X.reshape(train_dataset.X.shape[0], -1), train_dataset.y
        X_v, y_v = val_dataset.X.reshape(val_dataset.X.shape[0], -1), val_dataset.y
        X_d, y_d = test_dataset_d.X.reshape(test_dataset_d.X.shape[0], -1), test_dataset_d.y
        X_nd, y_nd = test_dataset_nd.X.reshape(test_dataset_nd.X.shape[0], -1), test_dataset_nd.y


        # tune only once
        if s == seedlist[0]:

            # Support vector
            def objective(trial):
                params = {
                "C": trial.suggest_float("C", 0.2, 2.0, log=True),
                'kernel' : trial.suggest_categorical('kernel', ['rbf', 'linear', 'poly', 'sigmoid']), 
                'degree' : trial.suggest_int('degree', 2, 3),
                'gamma' : trial.suggest_categorical('gamma', ['auto', 'scale'])
                }

                clf = SVR(**params)
                clf.fit(X_t, y_t)

                # intermediate_value = clf.score(X_v, y_v)
                y_p = clf.predict(X_v)
                intermediate_value =  mean_squared_error(y_v, y_p)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                return intermediate_value

            # optimize study
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=num_trial) 
            print(study.best_params)
            
            # save study
            study_df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
            study_df.to_csv(os.path.join(output_dir, 'study_params_'+'{}.csv'.format(exp_name)))
            

            # dump best parameters
            exp_params[exp_name] = study.best_params


        # use best params from tuning
        rf_opt = SVR(**study.best_params)
        rf_opt.fit(X_t, y_t)

        # predict training year
        y_pred_t = rf_opt.predict(X_t)

        # predict validation year
        y_pred_v = rf_opt.predict(X_v)

        #predict drought year
        y_pred_d = rf_opt.predict(X_d)
        pkl.dump(y_d, open(os.path.join(output_dir, 'y_true_test_data_{}.pkl'.format(test_years_d[0])), 'wb'))
        pkl.dump(y_pred_d, open(os.path.join(output_dir, 'y_pred_test_data_{}.pkl'.format(test_years_d[0])), 'wb'))


        #predict non-drought year
        y_pred_nd = rf_opt.predict(X_nd)
        pkl.dump(y_nd, open(os.path.join(output_dir, 'y_true_test_data_{}.pkl'.format(test_years_nd[0])), 'wb'))
        pkl.dump(y_pred_nd, open(os.path.join(output_dir, 'y_pred_test_data_{}.pkl'.format(test_years_nd[0])), 'wb'))


        # print('training metrics')
        mape, rmse, r2, r = evalMetrics(y_t, y_pred_t)
        metrics_train.append([exp_name, 'train', 0, str(s), mape, rmse, r2, r])
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))

        # print('training metrics')
        mape, rmse, r2, r = evalMetrics(y_v, y_pred_v)
        metrics_val.append([exp_name, 'validation', 0, str(s), mape, rmse, r2, r])
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))


        # # print('validation metrics d')
        mape, rmse, r2, r = evalMetrics(y_d, y_pred_d)
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))
        metrics_test.append([ exp_name,'test_d', test_years_d[0], str(s), mape, rmse, r2, r])

        # print('validation metrics nd')
        mape, rmse, r2, r = evalMetrics(y_nd, y_pred_nd)
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))
        metrics_test.append([ exp_name,'test nd', test_years_nd[0], str(s), mape, rmse, r2, r])


    
    results_df_train = pd.DataFrame(metrics_train, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    results_df_val = pd.DataFrame(metrics_val, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    results_df_test = pd.DataFrame(metrics_test, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    df_combined = pd.concat([results_df_train, results_df_val, results_df_test], ignore_index=True)
    df_combined.to_csv(os.path.join(output_dir, '{}.csv'.format(exp_name)))

    # save exp parameters
    with open(os.path.join(output_dir, 'params.json'), 'w') as file:
        file.write(json.dumps(exp_params, indent=4))

    # save geoid
    pkl.dump(test_dataset_d.geoid , open(os.path.join(output_dir, 'geoid_{}.pkl'.format(test_years_d[0])), 'wb'))
    pkl.dump(test_dataset_nd.geoid , open(os.path.join(output_dir, 'geoid_{}.pkl'.format(test_years_nd[0])), 'wb'))
        
    print(exp_name, 'done')

In [8]:
output_dir = '/app/dev/spatial_encoding/2024_08/results'              

seedlist = [3407, 42, 1234]
num_trial=20

svr_train(seedlist, num_trial, output_dir, exp_name='svr')

[I 2024-09-04 06:54:30,827] A new study created in memory with name: no-name-9db4ac7a-ce68-4b03-b521-29ce81ede602
[I 2024-09-04 06:54:33,730] Trial 0 finished with value: 869.5141525746448 and parameters: {'C': 0.842720166836167, 'kernel': 'sigmoid', 'degree': 2, 'gamma': 'scale'}. Best is trial 0 with value: 869.5141525746448.
[I 2024-09-04 06:54:35,664] Trial 1 finished with value: 225.56470918691528 and parameters: {'C': 0.3769908876812393, 'kernel': 'linear', 'degree': 2, 'gamma': 'auto'}. Best is trial 1 with value: 225.56470918691528.
[I 2024-09-04 06:54:38,570] Trial 2 finished with value: 851.1953817996629 and parameters: {'C': 1.9984728888970515, 'kernel': 'sigmoid', 'degree': 2, 'gamma': 'scale'}. Best is trial 1 with value: 225.56470918691528.
[I 2024-09-04 06:54:40,509] Trial 3 finished with value: 238.64670182125488 and parameters: {'C': 0.27135359424122035, 'kernel': 'linear', 'degree': 3, 'gamma': 'auto'}. Best is trial 1 with value: 225.56470918691528.
[I 2024-09-04 06:

{'C': 1.9885572128091131, 'kernel': 'linear', 'degree': 3, 'gamma': 'auto'}
svr done


## random forest block

In [5]:
def rf_train(seedlist, num_trial, out_dir, exp_name='rf'):
    
    # prepare output_dir 
    os.makedirs(out_dir, exist_ok=True)
    exp_params = {}
    
    # make directory for experiment
    output_dir = os.path.join(out_dir, exp_name)
    os.makedirs(output_dir, exist_ok=True)

    # -- collector for metrics
    metrics_train = []
    metrics_val = []
    metrics_test = []   

    for s in seedlist:
        np.random.seed(s)
        random.seed(s)
        torch.manual_seed(s)

        # Initialize YieldDataset with various parameters
        train_dataset = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=train_years, mode='train', seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # Initialize YieldDataset with various parameters
        val_dataset = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=train_years, mode='validation', seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # Initialize YieldDataset with various parameters
        test_dataset_d = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=test_years_d, mode=None, seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        test_dataset_nd = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=test_years_nd, mode=None, seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # ----- create array for experiment settings
        X_t, y_t  = train_dataset.X.reshape(train_dataset.X.shape[0], -1), train_dataset.y
        X_v, y_v = val_dataset.X.reshape(val_dataset.X.shape[0], -1), val_dataset.y
        X_d, y_d = test_dataset_d.X.reshape(test_dataset_d.X.shape[0], -1), test_dataset_d.y
        X_nd, y_nd = test_dataset_nd.X.reshape(test_dataset_nd.X.shape[0], -1), test_dataset_nd.y


        # tune only once
        if s == seedlist[0]:


            def objective(trial):

                params = {
                'n_estimators' : trial.suggest_int('n_estimators', 100, 500),
                'max_depth' : trial.suggest_int('max_depth', 2, 10),
                # 'max_features' : trial.suggest_categorical('max_features', ['auto', 'sqrt']), 
                'min_samples_split' : trial.suggest_int('min_samples_split', 3, 10),
                'bootstrap' : trial.suggest_categorical('bootstrap', [True, False]),
                'n_jobs' : trial.suggest_categorical('n_jobs', [-1]), 
                'random_state' : trial.suggest_categorical('random_state', [s])
                }

                clf = RandomForestRegressor(**params)
                clf.fit(X_t, y_t)

                # intermediate_value = clf.score(X_v, y_v)
                y_p = clf.predict(X_v)
                intermediate_value =  mean_squared_error(y_v, y_p)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                return intermediate_value
            
            # optimize study
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=num_trial) 
            print(study.best_params)
            
            # save study
            study_df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
            study_df.to_csv(os.path.join(output_dir, 'study_params_'+'{}.csv'.format(exp_name)))
            

            # dump best parameters
            exp_params[exp_name] = study.best_params


        # use best params from tuning
        rf_opt = RandomForestRegressor(**study.best_params)
        rf_opt.fit(X_t, y_t)

        # predict training year
        y_pred_t = rf_opt.predict(X_t)

        # predict validation year
        y_pred_v = rf_opt.predict(X_v)

        #predict drought year
        y_pred_d = rf_opt.predict(X_d)
        pkl.dump(y_d, open(os.path.join(output_dir, 'y_true_test_data_{}.pkl'.format(test_years_d[0])), 'wb'))
        pkl.dump(y_pred_d, open(os.path.join(output_dir, 'y_pred_test_data_{}.pkl'.format(test_years_d[0])), 'wb'))


        #predict non-drought year
        y_pred_nd = rf_opt.predict(X_nd)
        pkl.dump(y_nd, open(os.path.join(output_dir, 'y_true_test_data_{}.pkl'.format(test_years_nd[0])), 'wb'))
        pkl.dump(y_pred_nd, open(os.path.join(output_dir, 'y_pred_test_data_{}.pkl'.format(test_years_nd[0])), 'wb'))


        # print('training metrics')
        mape, rmse, r2, r = evalMetrics(y_t, y_pred_t)
        metrics_train.append([exp_name, 'train', 0, str(s), mape, rmse, r2, r])
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))

        # print('training metrics')
        mape, rmse, r2, r = evalMetrics(y_v, y_pred_v)
        metrics_val.append([exp_name, 'validation', 0, str(s), mape, rmse, r2, r])
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))


        # # print('validation metrics d')
        mape, rmse, r2, r = evalMetrics(y_d, y_pred_d)
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))
        metrics_test.append([ exp_name,'test_d', test_years_d[0], str(s), mape, rmse, r2, r])

        # print('validation metrics nd')
        mape, rmse, r2, r = evalMetrics(y_nd, y_pred_nd)
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))
        metrics_test.append([ exp_name,'test nd', test_years_nd[0], str(s), mape, rmse, r2, r])


    
    results_df_train = pd.DataFrame(metrics_train, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    results_df_val = pd.DataFrame(metrics_val, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    results_df_test = pd.DataFrame(metrics_test, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    df_combined = pd.concat([results_df_train, results_df_val, results_df_test], ignore_index=True)
    df_combined.to_csv(os.path.join(output_dir, '{}.csv'.format(exp_name)))

    # save exp parameters
    with open(os.path.join(output_dir, 'params.json'), 'w') as file:
        file.write(json.dumps(exp_params, indent=4))

    # save geoid
    pkl.dump(test_dataset_d.geoid , open(os.path.join(output_dir, 'geoid_{}.pkl'.format(test_years_d[0])), 'wb'))
    pkl.dump(test_dataset_nd.geoid , open(os.path.join(output_dir, 'geoid_{}.pkl'.format(test_years_nd[0])), 'wb'))
        
    print(exp_name, 'done')

In [6]:
output_dir = '/app/dev/spatial_encoding/2024_08/results'              

seedlist = [3407, 42, 1234]
num_trial=20

rf_train(seedlist, num_trial, output_dir, exp_name='rf2')

[I 2024-09-04 07:15:55,652] A new study created in memory with name: no-name-77e5c5d3-e62c-4dc9-898c-748a992b1298
[I 2024-09-04 07:15:58,349] Trial 0 finished with value: 145.28029711661642 and parameters: {'n_estimators': 197, 'max_depth': 7, 'min_samples_split': 6, 'bootstrap': True, 'n_jobs': -1, 'random_state': 3407}. Best is trial 0 with value: 145.28029711661642.
[I 2024-09-04 07:16:01,748] Trial 1 finished with value: 193.33745255818963 and parameters: {'n_estimators': 325, 'max_depth': 5, 'min_samples_split': 5, 'bootstrap': True, 'n_jobs': -1, 'random_state': 3407}. Best is trial 0 with value: 145.28029711661642.
[I 2024-09-04 07:16:02,519] Trial 2 finished with value: 371.05561307417145 and parameters: {'n_estimators': 135, 'max_depth': 2, 'min_samples_split': 3, 'bootstrap': True, 'n_jobs': -1, 'random_state': 3407}. Best is trial 0 with value: 145.28029711661642.
[I 2024-09-04 07:16:05,949] Trial 3 finished with value: 231.80050661022386 and parameters: {'n_estimators': 392

{'n_estimators': 253, 'max_depth': 10, 'min_samples_split': 7, 'bootstrap': True, 'n_jobs': -1, 'random_state': 3407}
rf2 done


## xgboost

In [8]:
def xgb_train(seedlist, num_trial, out_dir, exp_name='xgb'):
    
    # prepare output_dir 
    os.makedirs(out_dir, exist_ok=True)
    exp_params = {}
    
    # make directory for experiment
    output_dir = os.path.join(out_dir, exp_name)
    os.makedirs(output_dir, exist_ok=True)

    # -- collector for metrics
    metrics_train = []
    metrics_val = []
    metrics_test = []   

    for s in seedlist:
        np.random.seed(s)
        random.seed(s)
        torch.manual_seed(s)

        # Initialize YieldDataset with various parameters
        train_dataset = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=train_years, mode='train', seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # Initialize YieldDataset with various parameters
        val_dataset = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=train_years, mode='validation', seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # Initialize YieldDataset with various parameters
        test_dataset_d = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=test_years_d, mode=None, seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        test_dataset_nd = YDataset(npy_path, label_path, norm_path= norm_path, 
                        lookup=test_years_nd, mode=None, seed=s, start_doy_idx=start_doy_idx, 
                        end_doy_idx=end_doy_idx, ignore_features= [9,10,11], 
                        kernel=kernel, feature_idx =feature_idx, 
                        smooth = True)

        # ----- create array for experiment settings
        X_t, y_t  = train_dataset.X.reshape(train_dataset.X.shape[0], -1), train_dataset.y
        X_v, y_v = val_dataset.X.reshape(val_dataset.X.shape[0], -1), val_dataset.y
        X_d, y_d = test_dataset_d.X.reshape(test_dataset_d.X.shape[0], -1), test_dataset_d.y
        X_nd, y_nd = test_dataset_nd.X.reshape(test_dataset_nd.X.shape[0], -1), test_dataset_nd.y
    



        # tune only once
        if s == seedlist[0]:

            def objective(trial):
                params = {
                    "verbosity": 0,
                    "objective": "reg:squarederror",
                    # use exact for small dataset.
                    # "tree_method": "exact",
                    # defines booster, gblinear for linear functions.
                    "booster": trial.suggest_categorical("booster", ["gbtree"]), #, "gblinear", "dart"
                    # L2 regularization weight.
                    "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
                    # L1 regularization weight.
                    "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
                    # sampling ratio for training data.
                    "subsample": trial.suggest_float("subsample", 0.6, 0.8), #0.6, 0.8
                    # sampling according to each tree.
                    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.5), #0.6, 0.8
                    'n_jobs' : trial.suggest_categorical('n_jobs', [30])
                }

                if params["booster"] in ["gbtree", "dart"]:
                    # number of estimators
                    params['n_estimators'] = trial.suggest_int('n_estimators', 100, 300)
                    # maximum depth of the tree, signifies complexity of the tree.
                    params["max_depth"] = trial.suggest_int("max_depth", 2, 5, step=1) 
                    # minimum child weight, larger the term more conservative the tree.
                    params["min_child_weight"] = trial.suggest_int("min_child_weight", 3, 8)
                    params["eta"] = trial.suggest_float("eta", 1e-4, 1e-2, log=True)
                    # # defines how selective algorithm is.
                    # params["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
                    params["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

#                     if params["booster"] == "dart":
#                         params["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
#                         params["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
#                         # params["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
#                         # params["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)


                clf = xgb.XGBRegressor(**params)
                clf.fit(X_t, y_t, eval_set=[(X_v, y_v)], verbose=True)

                # intermediate_value = clf.score(X_v, y_v)
                y_p = clf.predict(X_v)
                intermediate_value =  mean_squared_error(y_v, y_p)

                if trial.should_prune():
                    raise optuna.TrialPruned()

                return intermediate_value

            # optimize study
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=num_trial, timeout=600) 
            print(study.best_params)
            
            # save study
            study_df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
            study_df.to_csv(os.path.join(output_dir, 'study_params_'+'{}.csv'.format(exp_name)))
            

            # dump best parameters
            exp_params[exp_name] = study.best_params



        xg_opt = xgb.XGBRegressor(**study.best_params)
        xg_opt.fit(X_t, y_t)

        # predict training year
        y_pred_t = xg_opt.predict(X_t)

        # predict validation year
        y_pred_v = xg_opt.predict(X_v)

        #predict drought year
        y_pred_d = xg_opt.predict(X_d)
        pkl.dump(y_d, open(os.path.join(output_dir, 'y_true_test_data_{}.pkl'.format(test_years_d[0])), 'wb'))
        pkl.dump(y_pred_d, open(os.path.join(output_dir, 'y_pred_test_data_{}.pkl'.format(test_years_d[0])), 'wb'))


        #predict non-drought year
        y_pred_nd = xg_opt.predict(X_nd)
        pkl.dump(y_nd, open(os.path.join(output_dir, 'y_true_test_data_{}.pkl'.format(test_years_nd[0])), 'wb'))
        pkl.dump(y_pred_nd, open(os.path.join(output_dir, 'y_pred_test_data_{}.pkl'.format(test_years_nd[0])), 'wb'))


        # print('training metrics')
        mape, rmse, r2, r = evalMetrics(y_t, y_pred_t)
        metrics_train.append([exp_name, 'train', 0, str(s), mape, rmse, r2, r])
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))

        # print('training metrics')
        mape, rmse, r2, r = evalMetrics(y_v, y_pred_v)
        metrics_val.append([exp_name, 'validation', 0, str(s), mape, rmse, r2, r])
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))


        # # print('validation metrics d')
        mape, rmse, r2, r = evalMetrics(y_d, y_pred_d)
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))
        metrics_test.append([ exp_name,'test_d', test_years_d[0], str(s), mape, rmse, r2, r])

        # print('validation metrics nd')
        mape, rmse, r2, r = evalMetrics(y_nd, y_pred_nd)
        # print('MAPE= {} , RMSE = {} , r2 ={} , R = {}'.format(mape, rmse, r2, r))
        metrics_test.append([ exp_name,'test nd', test_years_nd[0], str(s), mape, rmse, r2, r])


    
    results_df_train = pd.DataFrame(metrics_train, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    results_df_val = pd.DataFrame(metrics_val, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    results_df_test = pd.DataFrame(metrics_test, columns=['exp_name', 'mode', 'year','seed', 'MAPE', 'RMSE', 'R2', 'r'])
    df_combined = pd.concat([results_df_train, results_df_val, results_df_test], ignore_index=True)
    df_combined.to_csv(os.path.join(output_dir, '{}.csv'.format(exp_name)))

    # save exp parameters
    with open(os.path.join(output_dir, 'params.json'), 'w') as file:
        file.write(json.dumps(exp_params, indent=4))

    # save geoid
    pkl.dump(test_dataset_d.geoid , open(os.path.join(output_dir, 'geoid_{}.pkl'.format(test_years_d[0])), 'wb'))
    pkl.dump(test_dataset_nd.geoid , open(os.path.join(output_dir, 'geoid_{}.pkl'.format(test_years_nd[0])), 'wb'))
        
    print(exp_name, 'done')

In [9]:
output_dir = '/app/dev/spatial_encoding/2024_08/results'              

seedlist = [3407, 42, 1234]
num_trial=20

xgb_train(seedlist, num_trial, output_dir, exp_name='xgb_3')

[I 2024-09-04 07:19:05,668] A new study created in memory with name: no-name-dbdb604d-2720-4db2-bf35-9194f75313c4


[0]	validation_0-rmse:29.79261
[1]	validation_0-rmse:29.77793
[2]	validation_0-rmse:29.76299
[3]	validation_0-rmse:29.74747
[4]	validation_0-rmse:29.73317
[5]	validation_0-rmse:29.71836
[6]	validation_0-rmse:29.70410
[7]	validation_0-rmse:29.68904
[8]	validation_0-rmse:29.67469
[9]	validation_0-rmse:29.66002
[10]	validation_0-rmse:29.64498
[11]	validation_0-rmse:29.63013
[12]	validation_0-rmse:29.61524
[13]	validation_0-rmse:29.60183
[14]	validation_0-rmse:29.58709
[15]	validation_0-rmse:29.57237
[16]	validation_0-rmse:29.55833
[17]	validation_0-rmse:29.54356
[18]	validation_0-rmse:29.52989
[19]	validation_0-rmse:29.51575
[20]	validation_0-rmse:29.50101
[21]	validation_0-rmse:29.48622
[22]	validation_0-rmse:29.47195
[23]	validation_0-rmse:29.45825
[24]	validation_0-rmse:29.44384
[25]	validation_0-rmse:29.42882
[26]	validation_0-rmse:29.41428
[27]	validation_0-rmse:29.39975
[28]	validation_0-rmse:29.38556
[29]	validation_0-rmse:29.37076
[30]	validation_0-rmse:29.35626
[31]	validation_0-

[I 2024-09-04 07:19:08,091] Trial 0 finished with value: 683.5522454455919 and parameters: {'booster': 'gbtree', 'lambda': 0.12189220735589204, 'alpha': 0.21995345897476404, 'subsample': 0.7885410333402273, 'colsample_bytree': 0.3717752337812597, 'n_jobs': 30, 'n_estimators': 277, 'max_depth': 4, 'min_child_weight': 6, 'eta': 0.000754809301343299, 'grow_policy': 'depthwise'}. Best is trial 0 with value: 683.5522454455919.


[0]	validation_0-rmse:29.72538
[1]	validation_0-rmse:29.64547
[2]	validation_0-rmse:29.56206
[3]	validation_0-rmse:29.47791
[4]	validation_0-rmse:29.40093
[5]	validation_0-rmse:29.31869
[6]	validation_0-rmse:29.24297
[7]	validation_0-rmse:29.16076
[8]	validation_0-rmse:29.08092
[9]	validation_0-rmse:29.00770
[10]	validation_0-rmse:28.92649
[11]	validation_0-rmse:28.84668
[12]	validation_0-rmse:28.76857
[13]	validation_0-rmse:28.69645
[14]	validation_0-rmse:28.61765
[15]	validation_0-rmse:28.53679
[16]	validation_0-rmse:28.46613
[17]	validation_0-rmse:28.39055
[18]	validation_0-rmse:28.31928
[19]	validation_0-rmse:28.24266
[20]	validation_0-rmse:28.16522
[21]	validation_0-rmse:28.09105
[22]	validation_0-rmse:28.01756
[23]	validation_0-rmse:27.95100
[24]	validation_0-rmse:27.87583
[25]	validation_0-rmse:27.80000
[26]	validation_0-rmse:27.72807
[27]	validation_0-rmse:27.65453
[28]	validation_0-rmse:27.58167
[29]	validation_0-rmse:27.50967
[30]	validation_0-rmse:27.43954
[31]	validation_0-

[I 2024-09-04 07:19:09,864] Trial 1 finished with value: 333.3888720508462 and parameters: {'booster': 'gbtree', 'lambda': 0.010464083689380104, 'alpha': 3.6897447161452e-07, 'subsample': 0.679529343918663, 'colsample_bytree': 0.39209706484854806, 'n_jobs': 30, 'n_estimators': 242, 'max_depth': 3, 'min_child_weight': 8, 'eta': 0.00470456886008781, 'grow_policy': 'lossguide'}. Best is trial 1 with value: 333.3888720508462.


...

[0]	validation_0-rmse:29.76653
[1]	validation_0-rmse:29.72621
[2]	validation_0-rmse:29.68483
[3]	validation_0-rmse:29.64371
[4]	validation_0-rmse:29.60560
[5]	validation_0-rmse:29.56509
[6]	validation_0-rmse:29.52740
[7]	validation_0-rmse:29.48597
[8]	validation_0-rmse:29.44545
[9]	validation_0-rmse:29.40761
[10]	validation_0-rmse:29.36626
[11]	validation_0-rmse:29.32479
[12]	validation_0-rmse:29.28422
[13]	validation_0-rmse:29.24675
[14]	validation_0-rmse:29.20679
[15]	validation_0-rmse:29.16727
[16]	validation_0-rmse:29.13073
[17]	validation_0-rmse:29.09096
[18]	validation_0-rmse:29.05428
[19]	validation_0-rmse:29.01759
[20]	validation_0-rmse:28.97885
[21]	validation_0-rmse:28.94055
[22]	validation_0-rmse:28.90164
[23]	validation_0-rmse:28.86764
[24]	validation_0-rmse:28.82814
[25]	validation_0-rmse:28.78878
[26]	validation_0-rmse:28.75050
[27]	validation_0-rmse:28.71171
[28]	validation_0-rmse:28.67250
[29]	validation_0-rmse:28.63523
[30]	validation_0-rmse:28.59674
[31]	validation_0-

[I 2024-09-04 07:19:41,634] Trial 18 finished with value: 564.2653428278401 and parameters: {'booster': 'gbtree', 'lambda': 0.01794834184890126, 'alpha': 0.033399514566515145, 'subsample': 0.6003604414846564, 'colsample_bytree': 0.3667668385081121, 'n_jobs': 30, 'n_estimators': 187, 'max_depth': 3, 'min_child_weight': 4, 'eta': 0.0023262898796230958, 'grow_policy': 'lossguide'}. Best is trial 12 with value: 197.61703704472308.


[0]	validation_0-rmse:29.69095
[1]	validation_0-rmse:29.57332
[2]	validation_0-rmse:29.45521
[3]	validation_0-rmse:29.33340
[4]	validation_0-rmse:29.22017
[5]	validation_0-rmse:29.10491
[6]	validation_0-rmse:28.98805
[7]	validation_0-rmse:28.87212
[8]	validation_0-rmse:28.75897
[9]	validation_0-rmse:28.64939
[10]	validation_0-rmse:28.53356
[11]	validation_0-rmse:28.42174
[12]	validation_0-rmse:28.31087
[13]	validation_0-rmse:28.20701
[14]	validation_0-rmse:28.09769
[15]	validation_0-rmse:27.99342
[16]	validation_0-rmse:27.89192
[17]	validation_0-rmse:27.78641
[18]	validation_0-rmse:27.68358
[19]	validation_0-rmse:27.58451
[20]	validation_0-rmse:27.47864
[21]	validation_0-rmse:27.37476
[22]	validation_0-rmse:27.27228
[23]	validation_0-rmse:27.17413
[24]	validation_0-rmse:27.07200
[25]	validation_0-rmse:26.96586
[26]	validation_0-rmse:26.86533
[27]	validation_0-rmse:26.76393
[28]	validation_0-rmse:26.66322
[29]	validation_0-rmse:26.56213
[30]	validation_0-rmse:26.46217
[31]	validation_0-

[I 2024-09-04 07:19:44,936] Trial 19 finished with value: 215.91689079497448 and parameters: {'booster': 'gbtree', 'lambda': 0.00030548461775321996, 'alpha': 0.016041956970801474, 'subsample': 0.6664236376810446, 'colsample_bytree': 0.3514398938132568, 'n_jobs': 30, 'n_estimators': 263, 'max_depth': 4, 'min_child_weight': 7, 'eta': 0.0060816740120911246, 'grow_policy': 'lossguide'}. Best is trial 12 with value: 197.61703704472308.


{'booster': 'gbtree', 'lambda': 0.0006479776917988095, 'alpha': 7.071310990057505e-05, 'subsample': 0.6013904066297826, 'colsample_bytree': 0.3309432759813565, 'n_jobs': 30, 'n_estimators': 240, 'max_depth': 3, 'min_child_weight': 7, 'eta': 0.009953987535067708, 'grow_policy': 'lossguide'}
xgb_3 done
